In [34]:
# example of saving models for a snapshot ensemble
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor

# snapshot ensemble with custom learning rate schedule
class SnapshotEnsemble(Callback):
	# constructor
	def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
		self.epochs = n_epochs
		self.cycles = n_cycles
		self.lr_max = lrate_max
		self.lrates = list()

	# calculate learning rate for epoch
	def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
		epochs_per_cycle = floor(n_epochs/n_cycles)
		cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
		return lrate_max/2 * (cos(cos_inner) + 1)

	# calculate and set learning rate at the start of the epoch
	def on_epoch_begin(self, epoch, logs={}):
		# calculate learning rate
		lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
		# set learning rate
		backend.set_value(self.model.optimizer.lr, lr)
		# log value
		self.lrates.append(lr)

	# save models at the end of each cycle
	def on_epoch_end(self, epoch, logs={}):
		# check if we can save model
		epochs_per_cycle = floor(self.epochs / self.cycles)
		if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
			# save model to file
			filename = "snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
			self.model.save(filename)
			print('>saved snapshot %s, epoch %d' % (filename, epoch))

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)
# one hot encode output variable
y = to_categorical(y)
# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]
# define model
model = Sequential()
model.add(Dense(50, input_dim=2, activation='relu'))
model.add(Dense(3, activation='softmax'))
opt = SGD(momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# create snapshot ensemble callback
n_epochs = 500
n_cycles = n_epochs / 50
ca = SnapshotEnsemble(n_epochs, n_cycles, 0.01)
# fit model
model.fit(trainX, trainy, validation_data=(testX, testy), epochs=n_epochs, verbose=0, callbacks=[ca])

>saved snapshot snapshot_model_1.h5, epoch 49
>saved snapshot snapshot_model_2.h5, epoch 99
>saved snapshot snapshot_model_3.h5, epoch 149
>saved snapshot snapshot_model_4.h5, epoch 199
>saved snapshot snapshot_model_5.h5, epoch 249
>saved snapshot snapshot_model_6.h5, epoch 299
>saved snapshot snapshot_model_7.h5, epoch 349
>saved snapshot snapshot_model_8.h5, epoch 399
>saved snapshot snapshot_model_9.h5, epoch 449
>saved snapshot snapshot_model_10.h5, epoch 499


In [35]:
testX.shape, testy.shape

((1000, 2), (1000, 3))

In [40]:
model.evaluate(testX, testy)

32/32 [==============================] - 0s 960us/step - loss: 0.4751 - accuracy: 0.8100


[0.47510215640068054, 0.8100000023841858]

In [41]:
def load_models(num_models):
    models = []
    for i in range(num_models):
        models.append(keras.models.load_model("snapshot_model_" + str(i+1) + ".h5"))
    return models

In [55]:
def ensemble_predictions(models, testX):
    ans = []
    for model in models:
        ans.append(model.predict(testX))
    ans = np.array(ans).sum(axis=0).argmax(1)
    return ans

In [83]:
models = load_models(8)
pred = ensemble_predictions(models, testX)

32/32 [==============================] - 0s 857us/step


In [84]:
len(pred[pred==testy.argmax(1)])/len(pred)

0.814

In [60]:
testy.argmax(1)

array([0, 2, 2, 2, 0, 0, 1, 2, 1, 1, 2, 2, 1, 0, 2, 2, 1, 1, 0, 2, 2, 1,
       0, 1, 0, 0, 2, 1, 0, 1, 1, 2, 2, 0, 2, 0, 0, 2, 1, 1, 2, 2, 2, 2,
       2, 1, 0, 2, 0, 1, 1, 2, 0, 2, 2, 2, 0, 2, 1, 0, 2, 0, 0, 2, 1, 0,
       2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 1, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0,
       0, 1, 0, 1, 1, 0, 2, 0, 1, 1, 2, 2, 1, 2, 1, 1, 2, 0, 2, 0, 1, 2,
       1, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 2, 1, 0, 0, 2,
       1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1,
       1, 0, 2, 2, 1, 2, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 0, 0, 1, 1, 0,
       0, 0, 2, 1, 2, 0, 2, 0, 0, 1, 1, 2, 0, 1, 0, 0, 2, 0, 1, 2, 2, 0,
       2, 2, 0, 1, 1, 2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 1, 2, 2, 0, 2, 2,
       0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 0, 1, 1, 2, 1, 0, 1, 2, 2, 2, 2,
       2, 2, 2, 0, 1, 1, 0, 0, 2, 0, 2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 2,
       1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 1, 1,

In [47]:
import numpy as np

In [54]:
np.array(pred).sum(axis=0).argmax(1)

array([0, 2, 2, 2, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 2, 2, 1, 1, 0, 2, 2, 1,
       0, 1, 0, 0, 2, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 1, 1, 1, 1, 2, 0, 0,
       1, 1, 0, 2, 0, 1, 1, 2, 0, 2, 1, 2, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0,
       2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 1, 0, 0, 2, 0, 2, 2, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 2, 0, 2, 0, 0, 0,
       1, 2, 2, 0, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 2, 1, 2, 2, 0, 1, 1,
       1, 0, 0, 2, 1, 2, 1, 0, 0, 0, 1, 2, 2, 0, 2, 0, 0, 2, 2, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 0, 1, 0, 0, 2, 1, 0,
       0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 2, 2, 0, 1, 0, 0, 2, 0, 1, 2, 0, 0,
       2, 1, 0, 1, 1, 2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 1, 2,
       0, 1, 1, 1, 2, 0, 0, 1, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 2, 1,
       1, 2, 0, 2, 1, 1, 0, 0, 2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 1, 0, 1, 2,